1. First create a tmp folder
2. Then upload stopwords-bn.txt file


importing necessary modules

In [ ]:
import re
import nltk
import itertools
import collections
import editdistance
import networkx as nx
from sklearn.manifold import MDS
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Sentence Tokenizing from a text.

In [ ]:
class sentTokenizing:
    def sentTokenize(self,gettingText):
        dataToReSize=[]
        data=[]
        cleanText=''
        for i in gettingText:
            if i=='।' or i=='!' or i=='?':
                cleanText+=i
                dataToReSize.append(''.join(cleanText))
                cleanText=''
            else:
                if i=='\n' or i=='\r' or i=='”' or i=='“' or i=='"':
                    continue
                else:
                    cleanText+=i
        #print (dataToReSize)
        for i in dataToReSize:
            withoutAheadSpace=''
            flag=1
            for j in i:
                if j==' ' and flag:
                    continue
                else:
                    flag=0
                    withoutAheadSpace+=j
            data.append(''.join(withoutAheadSpace))
        #print(data)

        return data

WordConverter

In [ ]:
class word:
    def sentToWord(self,gettingData):
        cleanSent=''
        for i in gettingData:
            for j in i:
                if j=='”' or j=='“' or j=='"' or j==',' or j=='‘' or j=='’':
                    cleanSent+=' '
                    continue
                elif j=='!' or j=='?' or j=='।':
                    cleanSent+=' '
                    continue
                elif j=='(' or j=='{' or j=='}' or j=='[' or j==']':
                    cleanSent+=' '
                else:
                    if j=='-' or j==':' or j==')':
                        cleanSent+=' '
                    else:
                        cleanSent+=j
        return nltk.word_tokenize(cleanSent)

Textrank

In [ ]:
def build_graph(nodes):
    """Return a networkx graph instance.
    :param nodes: List of hashables that represent the nodes of a graph.
    """
    gr = nx.Graph()  # initialize an undirected graph
    gr.add_nodes_from(nodes)
    nodePairs = list(itertools.combinations(nodes, 2))

    # add edges to the graph (weighted by Levenshtein distance)
    for pair in nodePairs:
        firstString = pair[0]
        secondString = pair[1]
        levDistance = editdistance.eval(firstString, secondString)
        gr.add_edge(firstString, secondString, weight=levDistance)
    return gr

def extract_sentences(text, summary_length=40, clean_sentences=False, language='bangla'):
    """Return a paragraph formatted summary of the source text.
    :param text: A string.
    """
    sentence_tokens = sentTokenizing().sentTokenize(text)
    graph = build_graph(sentence_tokens)
    calculated_page_rank = nx.pagerank(graph, weight='weight')

    # most important sentences in ascending order of importance
    sentences = sorted(calculated_page_rank, key=calculated_page_rank.get, reverse=True)

    # return a 100 word summary
    summary = ' '.join(sentences)
    summary_words = summary.split()
    summary_words = summary_words[0:summary_length]
    dot_indices = [idx for idx, word in enumerate(summary_words) if word.find('।') != -1]
    #if clean_sentences and dot_indices:
    if  dot_indices:
        last_dot = max(dot_indices) + 1
        summary = ' '.join(summary_words[0:last_dot])
    else:
        summary = ' '.join(summary_words)

    return summary


def extract_summary(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        #summary = extract_key_phrases(f.read())
        summary = extract_sentences(f.read())
        return summary

Clustering

In [ ]:
def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Zঅ-ঔং ৎ  ক-‍ঁ ]', token):
            filtered_tokens.append(token)
    return filtered_tokens

def hierarchicalClustering(data,pos,document):
    key = 0
    k = []
    my_dict = dict()

    silhouette_scores = []
    for n in range(2,len(document)):
        #Find the agglomerativeClustering
        cluster1 = AgglomerativeClustering(n_clusters=n, linkage='ward')
        y1 = cluster1.fit_predict(pos)
        #Calculate Silhouette Scores
        silhouette_scores.append(silhouette_score(pos, y1))
        k.append(n)
        my_dict[key]= n
        key = key + 1
    #Find the maximum Silhouette Score
    maxx = silhouette_scores.index(max(silhouette_scores))

    # This section is intentionally commented out, uncomment this if you want to see the graph of Silhouette Scores.
    # Plotting a bar graph to compare the results
    # plt.bar(k, silhouette_scores)
    # plt.xlabel('Number of clusters', fontsize = 20)
    # plt.ylabel('S(i)', fontsize = 20)
    # plt.show()

    #Get the no of cluster which have maximum Silhouette Score
    val = list(my_dict.values())
    n_clusters = val[maxx]

    #Find the agglomerativeClustering
    cluster1 = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
    y1 = cluster1.fit_predict(pos)

    #Create file
    p = str(n_clusters)
    f = open("tmp/cluster"+p+".txt","w", encoding='utf-8')
    f.truncate(0)
    f = open("tmp/cluster"+p+".txt","a+", encoding='utf-8')

    #Save the clusters in the temporary file
    clusters = collections.defaultdict(list)
    for i, label in enumerate(cluster1.labels_):
        clusters[label].append(i)
    dict(clusters)

    for cluster in range(n_clusters):
        # print("cluster ",cluster,":")

        for i,sentence in enumerate(clusters[cluster]):
            # print("\tsentence ",i,": ",document[sentence])
            f.write(document[sentence])
            f.write(" ")
        f.write('\n\n')
    return n_clusters

def multidimensionalScaling(similarity_distance,document):
    mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)
    pos = mds.fit_transform(similarity_distance)  # shape (n_components, n_samples)
    xs, ys = pos[:, 0], pos[:, 1]
    n_clusters = hierarchicalClustering(similarity_distance,pos,document)
    return n_clusters

def similarityDistance(tfidf_matrix,document):
    similarity_distance = 1 - cosine_similarity(tfidf_matrix)
    n_clusters = multidimensionalScaling(similarity_distance,document)
    return n_clusters

def tfidf(document,stopword):
    tfidf_vectorizer = TfidfVectorizer(max_df=10, min_df=2, use_idf=True, tokenizer=tokenize_only, ngram_range=(1,3))
    tfidf_matrix = tfidf_vectorizer.fit_transform(document)
    terms = tfidf_vectorizer.get_feature_names_out()
    n_clusters = similarityDistance(tfidf_matrix,document)
    return n_clusters

def peep(n):
    p = str(n)
    cluster_sentence1 = open("tmp/cluster"+p+".txt", encoding='utf-8').read().split('\n\n')
    cluster_sentence1.pop((len(cluster_sentence1))-1)
    # print("Cluster Sentences are:",cluster_sentence1)
    filenamee1 = []
    for j in range(n):
        ab = str(j)
        namee = "tmp/Cluster"+p+"."+ab+".txt"
        filenamee1.append(namee)
        with open(namee, 'w', encoding='utf-8') as f:
            for item in cluster_sentence1[j]:
                f.write(item)
    return filenamee1, n

def startF(document):
    stopword = open('stopwords-bn.txt', 'r', encoding='utf-8').read().split('\n')
    n_clusters = tfidf(document,stopword)
    filenamee1, n  = peep(n_clusters)
    return filenamee1, n

Summary

In [ ]:
def listToString(s):
    str1 = " "
    return (str1.join(s))

#Sentence ordering
def sentence_ordering(unordered_summary):
    unordered_summary = [x[:(len(x)-1)].strip(' ') for x in unordered_summary]
    docs = [x[:(len(x)-1)].strip(' ') for x in doc]
    both = set(docs).intersection(unordered_summary)
    indices_A = [docs.index(x) for x in both]
    indices_B = [unordered_summary.index(x) for x in both]
    dictionary = {}
    for x in range(len(indices_A)):
        dictionary[indices_A[x]] = indices_B[x]
    ordered_summary = []
    for i in sorted (dictionary) :
        ordered_summary.append(dictionary[i])
    st =""
    for i in ordered_summary:
        st = st +(unordered_summary[i])+'।\n'
    return st

def getSummary(filename):
    summary = []
    for k in range(n):
        summary_lines = extract_summary(filename[k])
        summary.append(summary_lines)
    full_summary =[]
    for x in summary:
        left_text = x.partition("।")[0]
        left_text = left_text.partition("?")[0]
        left_text = left_text.partition("!")[0]
        full_summary.append(left_text+"।")
    s = listToString(full_summary)
    si = sentTokenizing().sentTokenize(s)
    # print(si)
    summ = sentence_ordering(si)
    return summ




# Summarize a single file

In [ ]:
# document = open('input.txt', 'r', encoding='utf-8').read()
document = "বেনাপোলের অবকাঠামোগত উন্নয়নসহ আমদানি-রপ্তানি বাণিজ্যের গতি ফেরাতে বেনাপোল-যশোর মহাসড়ক ৬ লেনে উন্নতকরাসহ বাইপাস সড়কের প্রশস্তকরণ, পণ্য লোড আনলোডের সরঞ্জাম এবং জনবল বৃদ্ধির লক্ষে রবিবার বিকেলে সড়ক ও জনপথসহ বিশ্ব ব্যাংকের ১৭ সদস্যের একটি প্রতিনিধিদল বেনাপোল বন্দর ও চেকপোস্ট এলাকা পরিদর্শন করেছেন।এর আগে বেনাপোল বন্দর প্যাসেজ্ঞার টার্মিনালের কনফারেন্স রুমে বন্দর, কাস্টম ও সিএন্ডএফ এজেন্ট ও বন্দর ব্যবহারকারী বিভিন্ন সংগঠনের নেতা ও প্রশাসনের কর্মকর্তাদের সাথে মত বিনিময় করেন প্রতিনিধিদলটি। উন্নয়ন কর্মকাণ্ডের পূর্ব প্রস্ততি হিসেবে প্রতিনিধি দলের বেনাপোল পরিদর্শন বলে জানান বন্দর সংশ্লিষ্টরা।বাংলাদেশ সড়ক ও জনপথের অতিরিক্ত প্রধান প্রকৌশলী রেজা আহম্মেদ জাবের প্রতিনিধিদলের নেতৃত্ব দেন। এ সময় উপস্থিত ছিলেন, বিশ্ব ব্যাংকের প্রতিনিধি মুন্নি জাহান, বাংলাদেশ স্থলবন্দর কর্তৃপক্ষের তত্বাবধায়ক প্রকৌশলী মোহাম্মদ হাসান আলী, স্থলবন্দর বেনাপোল উপপরিচালক মামুন কবির তরফদার, বেনাপোল কাস্টমের সহকারী কমিশনার উত্তম কুমার চাকমা, বন্দরের সহকারী পরিচালক মেহেদী হাসান, বেনাপোল সিএন্ডএফ এজেন্ট এসোসিয়েশন সভাপতি মফিজুর রহমান সজনসহ ভারত, নেপাল, নাইজেরিয়াসহ বিভিন্ন দেশের প্রতিনিধিরা।"
doc = sentTokenizing().sentTokenize(document)
filenamee, n = startF(doc)
summary = getSummary(filenamee)
print('Document : '+ document)
print('Summary : ' + summary)

Document : বেনাপোলের অবকাঠামোগত উন্নয়নসহ আমদানি-রপ্তানি বাণিজ্যের গতি ফেরাতে বেনাপোল-যশোর মহাসড়ক ৬ লেনে উন্নতকরাসহ বাইপাস সড়কের প্রশস্তকরণ, পণ্য লোড আনলোডের সরঞ্জাম এবং জনবল বৃদ্ধির লক্ষে রবিবার বিকেলে সড়ক ও জনপথসহ বিশ্ব ব্যাংকের ১৭ সদস্যের একটি প্রতিনিধিদল বেনাপোল বন্দর ও চেকপোস্ট এলাকা পরিদর্শন করেছেন।এর আগে বেনাপোল বন্দর প্যাসেজ্ঞার টার্মিনালের কনফারেন্স রুমে বন্দর, কাস্টম ও সিএন্ডএফ এজেন্ট ও বন্দর ব্যবহারকারী বিভিন্ন সংগঠনের নেতা ও প্রশাসনের কর্মকর্তাদের সাথে মত বিনিময় করেন প্রতিনিধিদলটি। উন্নয়ন কর্মকাণ্ডের পূর্ব প্রস্ততি হিসেবে প্রতিনিধি দলের বেনাপোল পরিদর্শন বলে জানান বন্দর সংশ্লিষ্টরা।বাংলাদেশ সড়ক ও জনপথের অতিরিক্ত প্রধান প্রকৌশলী রেজা আহম্মেদ জাবের প্রতিনিধিদলের নেতৃত্ব দেন। এ সময় উপস্থিত ছিলেন, বিশ্ব ব্যাংকের প্রতিনিধি মুন্নি জাহান, বাংলাদেশ স্থলবন্দর কর্তৃপক্ষের তত্বাবধায়ক প্রকৌশলী মোহাম্মদ হাসান আলী, স্থলবন্দর বেনাপোল উপপরিচালক মামুন কবির তরফদার, বেনাপোল কাস্টমের সহকারী কমিশনার উত্তম কুমার চাকমা, বন্দরের সহকারী পরিচালক মেহেদী হাসান, বেনাপোল সিএন্ডএফ এজেন্ট এসোসিয়েশন সভাপতি ম

# Summarize a csv file & save content summary into another csv file

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv('text_summarization_dataset2.csv')
summary_df = pd.DataFrame(columns=['content', 'summary'])

for i in range(0,df.size-1):
  tmp = open('tmp_input.txt', 'w', encoding='utf-8')
  content = df['content'][i]
  tmp.write(content)
  tmp.close()

  document = open('tmp_input.txt', 'r', encoding='utf-8').read()
  doc = sentTokenizing().sentTokenize(document)
  try:
    filenamee, n = startF(doc)

    summary = getSummary(filenamee)

    summary_df = summary_df.append({'content': content, 'summary': summary}, ignore_index=True)
  except:
    #continue
    summary = None

summary_df.to_csv('summary_dataset_2.csv')

KeyError: 105252

In [ ]:
summary_df.size


180622

In [ ]:
summary_df.to_csv('summary_dataset_2.csv')

In [ ]:
df['content'].size

105252

In [ ]:
import pandas as pd
df = pd.read_csv('data.csv')

# print(df.head())

import warnings
warnings.filterwarnings("ignore")

# for i in range(df.size):
print(df['content'][1])
doc = sentTokenizing().sentTokenize(df['content'][1])
print(doc)
filenamee, n = startF(doc)
summary = getSummary(filenamee)
print('summary: ', summary)

বাংলাদেশ প্রকৌশল বিশ্ববিদ্যালয় (বুয়েট) আজ বৃহস্পতিবার থেকে অনির্দিষ্টকালের জন্য বন্ধ ঘোষণা করা হয়েছে। আজ বিকেল পাঁচটার মধ্যে আবাসিক হলে অবস্থানরত সব ছাত্র-ছাত্রীকে হল ছাড়ার নির্দেশ দেওয়া হয়েছে। আজ আড়াইটার দিকে এ আদেশ সংবলিত বিজ্ঞপ্তি বিভিন্ন হলের নোটিশ বোর্ডে সেঁটে দেওয়া হয়। বুয়েটের রেজিস্ট্রার অধ্যাপক এ কে এম মাসুদ স্বাক্ষরিত ওই বিজ্ঞপ্তির ভাষ্য, চলতি টার্মের পূর্ব ঘোষিত টার্ম ফাইনাল পরীক্ষা পেছানোর দাবিতে ২৩ জুন একদল ছাত্র-ছাত্রীর উপাচার্য, রেজিস্ট্রার ও ছাত্রকল্যাণ পরিচালককে উপাচার্য কার্যালয়ে জিম্মি করা, ২৪ জুন শিক্ষকদের আবাসিক এলাকা অবরুদ্ধ করা এবং রাতে একাডেমিক ভবন ভাঙচুর ও অগ্নিসংযোগ করার পরিপ্রেক্ষিতে বিশ্ববিদ্যালয়ের সার্বিক আইনশৃঙ্খলা পরিস্থিতির চরম অবনতি ঘটছে এবং শিক্ষার পরিবেশ ভীষণভাবে বিঘ্নিত হচ্ছে। এ অবস্থায় বিশ্ববিদ্যালয়ের সার্বিক শৃঙ্খলা বজায় রাখা, ছাত্র-ছাত্রী, শিক্ষক ও কর্মকর্তা-কর্মচারীদের জানমালের নিরাপত্তা বিধানের স্বার্থে এবং শিক্ষার সুষ্ঠু পরিবেশ ফিরিয়ে আনার লক্ষ্যে এ বিশ্ববিদ্যালয়ের সব শিক্ষা কার্যক্রম আজ বিকেল থেকে অনির্দিষ্টকালের জন্য বন্ধ ঘোষণা করা হলো। রীক্ষা

In [ ]:
import pandas as pd
df = pd.read_csv('data.csv')
new_df = pd.DataFrame(columns=['content', 'summary', 'bensumm'])
for i in range(df.size):
  try:
    doc = sentTokenizing().sentTokenize(df['content'][i])
    filenamee, n = startF(doc)
    summary = getSummary(filenamee)
    new_df = new_df.append({'content': df['content'][i], 'summary': df['summary'][i], 'bensumm': summary}, ignore_index=True)
  except:
    summary = None

new_df.to_csv('bensumm_data.csv')